### Model Structure
| Layer             |
| ----------------- |
| `Conv2D(16, 3x3)` |
| `Conv2D(16, 3x3)` |
| `MaxPool2D(2x2)`  |
| `Dropout`         |
| `Conv2D(32, 3x3)` |
| `Conv2D(32, 3x3)` |
| `MaxPool2D(4x4)`  |
| `Dropout`         |
| `Flatten`         |
| `Dense(256)`      |
| `Dropout`         |
| `Dense(1)`        |

![BRAAI CNN Model Structure](images/fig-braai.png)

In [1]:
# Layer Class (template)
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, output_gradient, learning_rate):
        pass

In [ ]:
from scipy import signal
import numpy as np

class Conv2D(Layer):
    def __init__(self, input_shape, kernel_size, output_depth):
        input_depth, input_height, input_width = input_shape
        self.input_depth = input_depth
        self.input_shape = input_shape

        self.output_depth = output_depth
        self.output_shape = (output_depth, input_height-kernel_size+1, input_width-kernel_size+1)

        self.kernels_shape = (output_depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(self.output_depth)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.array(self.biases, copy=True)
        for i in range(self.output_depth):
            for j in range(self.input_depth):
                self.output = self.output + signal.correlate2d(self.input[j], self.kernels[i,j], "valid")

        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.output_depth):
            for j in range(self.input_depth):
                kernels_gradient[i,j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i,j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient.sum(axis=(1,2))
        return input_gradient